In [50]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")


In [108]:
# 设置数据集路径
Workspace = "/home/vscode/workspace"
DATA_PATH = "data/store/external/Data/tick"
DATA_PATH = os.path.join(Workspace, DATA_PATH)
DATA_PATH

'/home/vscode/workspace/data/store/external/Data/tick'

In [ ]:
def get_dt_stk_highfreq(date, table_name, code_list=None, key_list = None, print_col = False, 
                        path = f'{Workspace}/data/store/external/Data'):
    '''
    获取date日期股票的table_name的高频数据，该函数股票专用！
    '''
    date = pd.to_datetime(date).strftime("%Y%m%d")
    file = h5py.File(f'{path}/{table_name}/{date}.h5', 'r')
    if print_col:
        print(f'{table_name}有这些列：',file[list(file.keys())[0]].keys())  #打印高频数据有哪些
        return None
    res = []
    code_list = sorted(file.keys()) if code_list is None else [x for x in file.keys() if x.startswith(tuple(code_list))]
    for code in code_list:
        cd_dataset = file[code]
        if cd_dataset['Time'].shape == (0,): ##############有些停牌股票有键，但是shape为0########################
            continue                         ##############有些停牌股票有键，但是shape为0########################
        # cd_dataset = {key: cd_dataset[key] for key in cd_dataset} if key_list else {key: cd_dataset[key] for key in key_list}
        cd_dataset = {key: cd_dataset[key][()] for key in (key_list if key_list else cd_dataset.keys())}  # 和上一行实现功能相同
        #####################################################读取tick数据需要特别处理的##################################################
        for high_dim_col in (set(cd_dataset.keys()) & set(['AskPrice10','AskVolume10','BidPrice10','BidVolume10'])):  #高维数据需要展开
            for i in range(10):
                cd_dataset[f'{high_dim_col[:-2]}_{i+1}'] = cd_dataset[high_dim_col][:, i]
            del cd_dataset[high_dim_col]
        #####################################################读取tick数据需要特别处理的##################################################
        cd_dt_data = pd.DataFrame(cd_dataset)
        # cd_dt_data = cd_dt_data[cd_dt_data.Price != 0]
        cd_dt_data['code'] = code
        res.append(cd_dt_data)
    if len(res)==0:
        dt_highfreq = pd.DataFrame()
        print(f'{code_list}在{date}上缺失数据！')
    else:
        dt_highfreq = pd.concat(res).reset_index(drop=True)
    return dt_highfreq

In [126]:
dt_highfreq = get_dt_stk_highfreq('20251104', table_name='tick')

In [ ]:
dt_highfreq.shape

(26247505, 61)

In [131]:
dt_highfreq.columns

Index(['AccTurnover', 'AccVolume', 'AfterMatchItem', 'AfterPrice',
       'AfterTurnover', 'AfterVolume', 'AskAvgPrice', 'BSFlag', 'BidAvgPrice',
       'High', 'Low', 'MatchItem', 'Open', 'PreClose', 'Price', 'Time',
       'TotalAskVolume', 'TotalBidVolume', 'Turnover', 'Volume', 'BidPrice_1',
       'BidPrice_2', 'BidPrice_3', 'BidPrice_4', 'BidPrice_5', 'BidPrice_6',
       'BidPrice_7', 'BidPrice_8', 'BidPrice_9', 'BidPrice_10', 'AskVolume_1',
       'AskVolume_2', 'AskVolume_3', 'AskVolume_4', 'AskVolume_5',
       'AskVolume_6', 'AskVolume_7', 'AskVolume_8', 'AskVolume_9',
       'AskVolume_10', 'AskPrice_1', 'AskPrice_2', 'AskPrice_3', 'AskPrice_4',
       'AskPrice_5', 'AskPrice_6', 'AskPrice_7', 'AskPrice_8', 'AskPrice_9',
       'AskPrice_10', 'BidVolume_1', 'BidVolume_2', 'BidVolume_3',
       'BidVolume_4', 'BidVolume_5', 'BidVolume_6', 'BidVolume_7',
       'BidVolume_8', 'BidVolume_9', 'BidVolume_10', 'code'],
      dtype='object')

In [162]:
df = dt_highfreq.loc[(dt_highfreq['Time'] >= 100000000) & (dt_highfreq['Time'] <= 100300000)]
df = df[['Time',]]
df

SyntaxError: invalid syntax (628277064.py, line 2)

In [155]:
class TickFactorCalculator:
    def __init__(self, tick_data):
        """
        初始化tick因子计算器
        tick_data: DataFrame格式
        """
        self.tick_data = tick_data.copy()
        self.factors_data = None
        self._preprocess_data()
    def _preprocess_data(self):
        """数据预处理"""
        df = self.tick_data
        
        # 确保时间排序
        df = df.sort_values('Time').reset_index(drop=True)
        
        

10000    94503000
10001    94506000
10002    94509000
10003    94512000
10004    94515000
10005    94518000
10006    94521000
10007    94527000
10008    94530000
10009    94539000
10010    94542000
10011    94545000
10012    94548000
10013    94551000
10014    94554000
10015    94557000
10016    94600000
10017    94603000
10018    94606000
10019    94609000
Name: Time, dtype: uint32

In [163]:
df_minu = pd.read_parquet(Workspace+'/data/store/external/MintueData/TickData/Data/20251104.parquet')
df_minu.head(10)

,code,DateTime,volume,turnover,open,high,low,ret
0,000001.sz,20251104 92500,649600.0,7418432.0,-0.000875,-0.000875,-0.000875,-0.000875
1,000001.sz,20251104 93000,94827.0,1082765.0,0.000000,0.000000,0.000000,0.000000
2,000001.sz,20251104 93100,1494060.0,17065394.0,0.000000,0.002627,-0.001751,0.001751
3,000001.sz,20251104 93200,508400.0,5815127.0,0.000000,0.000000,-0.000874,0.000000
4,000001.sz,20251104 93300,556400.0,6367521.0,-0.000874,0.000874,-0.000874,0.000874
5,000001.sz,20251104 93400,776000.0,8880118.0,0.000000,0.000000,-0.000873,-0.000873
6,000001.sz,20251104 93500,763600.0,8737833.0,0.000000,0.000874,0.000000,0.000000
7,000001.sz,20251104 93600,1377028.0,15772754.0,0.000874,0.001748,0.000874,0.001748
8,000001.sz,20251104 93700,1020930.0,11710438.0,0.000873,0.001745,0.000000,0.000873
9,000001.sz,20251104 93800,796100.0,9134318.0,0.000000,0.000872,0.000000,0.000000


In [69]:
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y%m%d %H%M%S', errors='coerce')
df

,code,DateTime,open,high,low,close,BidPrice01,BidPrice02,BidPrice03,BidPrice04,...,AskVolume01,AskVolume02,AskVolume03,AskVolume04,AskVolume05,AskVolume06,AskVolume07,AskVolume08,AskVolume09,AskVolume10
0,000001.sz,2025-08-05 09:25:00,12.300000,12.300000,12.300000,12.300000,12.290000,12.280000,12.270000,12.260000,...,1700.0,134000.0,57840.0,122300.0,209000.0,665600.0,220500.0,293500.0,286800.0,339400.0
1,000001.sz,2025-08-05 09:30:00,12.320000,12.320000,12.320000,12.320000,12.310000,12.300000,12.290000,12.280000,...,58440.0,124300.0,211000.0,668300.0,221000.0,295500.0,289700.0,335400.0,858300.0,98000.0
2,000001.sz,2025-08-05 09:31:00,12.320000,12.320000,12.280000,12.290000,12.290000,12.280000,12.270000,12.260000,...,82200.0,124100.0,235740.0,335200.0,167400.0,801800.0,350800.0,342600.0,422500.0,393100.0
3,000001.sz,2025-08-05 09:32:00,12.290000,12.310000,12.280000,12.310000,12.300000,12.290000,12.280000,12.270000,...,179200.0,189740.0,350500.0,174000.0,809200.0,352800.0,357600.0,431200.0,402400.0,929700.0
4,000001.sz,2025-08-05 09:33:00,12.310000,12.320000,12.290000,12.320000,12.310000,12.300000,12.290000,12.280000,...,218740.0,297600.0,274100.0,590200.0,363400.0,357700.0,453600.0,409700.0,938100.0,156900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659873,689009.sh,2025-08-05 14:56:00,65.629997,65.629997,65.370003,65.370003,65.370003,65.349998,65.300003,65.260002,...,2300.0,2988.0,200.0,1000.0,675.0,1229.0,4534.0,1333.0,1779.0,4534.0
1659874,689009.sh,2025-08-05 14:57:00,65.379997,65.500000,65.199997,65.199997,65.209999,0.000000,0.000000,0.000000,...,402.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1659875,689009.sh,2025-08-05 14:58:00,65.199997,65.199997,65.199997,65.199997,65.209999,0.000000,0.000000,0.000000,...,61025.0,11405.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1659876,689009.sh,2025-08-05 14:59:00,65.199997,65.199997,65.199997,65.199997,65.199997,0.000000,0.000000,0.000000,...,146149.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def get_fields():
    fields = []  # 存储银子计算的表达式字符串
    names = []   # 存储对应因子的名称（用于标识因子）
    
    # 第一类因子：基于open、close、high、low计算得到的因子
    fileds += [
        "(close-open)/open",                              # 收盘价与开盘价的差值相对于开盘价的比例（涨跌幅度相对值）
        "(high-low)/open",                                # 最高价与最低价的差值相对于开盘价的比例（K线振幅相对值）
        "(close-open)/(high-low+1e-12)",                  # 涨跌额占振幅的比例（+1e-12避免分母为0）
        "(high-greater(open, close))/open",               # greater(a,b)返回a和b中的较大值；此处计算上影线长度/开盘价
        "(high-greater(open, close))/(high-low+1e-12)",   # 上影线长度占振幅的比例
        "(less(open, close)-low)/open",                   # less(a,b)返回a和b中的较小值；此处计算下影线长度/开盘价
        "(less(open, close)-low)/(high-low+1e-12)",       # 下影线长度占振幅的比例
        "(2*close-high-low)/open",                        # 收盘价偏离K线中点的程度（刻画K线重心）
        "(2*close-high-low)/(high-low+1e-12)",            # 收盘价偏离中点的程度占振幅的比例
    ]
    # 对应K线因子的名称（简洁标识因子含义）
    names += [
        "KMID",    # K线中点偏离度(close-open)/open
        "KLEN",    # K线振幅相对值 (high-low)/open
        "KMID2",   # 涨跌额占振幅比例 (close-open)/(high-low)
        "KUP",     # 上影线长度相对开盘价
        "KUP2",    # 上影线长度占振幅比例
        "KLOW",    # 下影线长度相对开盘价
        "KLOW2",   # 下影线长度占振幅比例
        "KSFT",    # K线重心偏移 (2*close-high-low)/open
        "KSFT2",   # 重心偏移占振幅比例
    ]